In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

import random

In [41]:
dataset = pd.read_csv(r"C:\Users\Siddhartha Devan V\Downloads\diabetes.csv")

In [42]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [44]:
dataset.Outcome.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [45]:
genes = dataset.columns
print(genes)

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')


In [46]:
dataset_1 = dataset[dataset["Outcome"] == 1].sample(270, replace = True)
dataset_0 = dataset[dataset["Outcome"] == 0].sample(270)

dataset_new = pd.concat((dataset_0, dataset_1), axis = "rows")

In [47]:
X = dataset_new[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']]

Y = dataset_new['Outcome']


In [48]:
Y.value_counts()

0    270
1    270
Name: Outcome, dtype: int64

In [62]:
class Gen_alg():
    def __init__(self, X, Y):
        self.features = X.columns
        self.Y = Y
        self.init_chrom = [0 for _ in range(len(X.columns))]
        self.chrom_len = len(X.columns)
        print("initial chromosome = ", self.init_chrom)
        self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(X, Y, train_size = 0.75, random_state = 0, stratify = Y)
        self.model = rf()
        self.best_off_springs = []
        
        """ In the __init__ function we initialize all the necessary variablessuch as the chromosome model,  train and test dataetc
        the parents list will contain the best child of every generation. The model is directly initialized as a random forest classifier"""
        
    def population_generator(self, pop_num  = 30):
        self.pop_num = pop_num
        self.population = []
        
        for _ in range(pop_num):
            chumma_chrom = self.init_chrom.copy()
            for i in range(len(chumma_chrom)):
                chumma_chrom[i] = random.choice([0,1])
            self.population.append(chumma_chrom)

            """ this function uses the initial chromosome as the template and generates a number of individual chromosomes and appends
            it to the population list. The alleles (0,1) are chosen randomly."""
            
    def feature_selector(self, chromosome):
#         print("chrome = ", chromosome)
        self.chrom_features = []
        for i in range(len(chromosome)):
            if chromosome[i] == 1:
                self.chrom_features.append(self.features[i])
        return self.chrom_features
        """This function selects the features of the chromosome. That is a chromosome of length eight would mean that there are 8 indpendent variables.
        if the gene of that particular atttriibute is 1, then the attribute is included and if it is 0, then it is omitted"""
        
    def fitness_func(self):
        self.feature_list = []
        self.fit_score = []
        for i in self.population:
            f = self.feature_selector(i)
            self.feature_list.append([i, f])
        
        for i in self.feature_list:
            chrom , feat = i
            X_train_temp = self.X_train[feat]
            X_test_temp = self.X_test[feat]
            
            self.model.fit(X_train_temp, self.Y_train)
            Y_pred_temp = self.model.predict(X_test_temp)
            self.fit_score.append(accuracy_score(Y_pred_temp, self.Y_test))
    
#         print(self.fit_score)
        
    """fitness function measures the fitness of a particular chromosome. Here fitness is measured in terms of accuracy of the model"""
    
    def select_best_pair(self):
        self.fitness_func()
        self.selected_chroms = []
        for i in range(10):
            selected_sc = max(self.fit_score)
            selected_ind = self.fit_score.index(selected_sc)
            self.fit_score.pop(selected_ind)
            self.selected = self.population.pop(selected_ind)
            self.selected_chroms.append([self.selected, selected_sc])
                
        self.best_off_springs.append(self.selected_chroms[0])
        """out if the population, top 10 individuals are considered to have suurvived and are selected"""
        
        
    def matter(self, n_child = 20):
        self.population = []
        for _ in range(n_child):
            mom = random.choice(self.selected_chroms)[0]
            dad = random.choice(self.selected_chroms)[0]
            child = []
            for i, j in zip(mom, dad):
                child.append(random.choice([i,j]))
            self.population.append(child)
            
        """the selected fit individuals are allowed to reproduce by crossing over and they self.population list is overwritten with the newly generated children.
            this process is done again and again to produce more generations. Eventually, since better offsprings areallowed to cross-over better chromosomes would be generated"""
            
    
    def generator(self, n_gen = 10):
        for _ in range(n_gen):
            self.matter()
            self.select_best_pair()
        
        for i in range(len(self.best_off_springs)):
            print("gen = ", i)
            print("best_off_spring of the generation is ", self.best_off_springs[i][0], self.best_off_springs[i][1])
            print("=="*30)
            print("--"*30)
            

In [63]:
obj = Gen_alg(X, Y)
obj.population_generator()
obj.select_best_pair()
obj.generator()

initial chromosome =  [0, 0, 0, 0, 0, 0, 0, 0]
gen =  0
best_off_spring of the generation is  [0, 1, 1, 1, 0, 0, 1, 1] 0.837037037037037
------------------------------------------------------------
gen =  1
best_off_spring of the generation is  [1, 1, 1, 1, 0, 0, 1, 1] 0.8222222222222222
------------------------------------------------------------
gen =  2
best_off_spring of the generation is  [1, 1, 1, 1, 1, 1, 1, 1] 0.8296296296296296
------------------------------------------------------------
gen =  3
best_off_spring of the generation is  [1, 1, 1, 1, 1, 1, 0, 1] 0.8222222222222222
------------------------------------------------------------
gen =  4
best_off_spring of the generation is  [1, 1, 1, 1, 0, 0, 1, 1] 0.8444444444444444
------------------------------------------------------------
gen =  5
best_off_spring of the generation is  [1, 1, 1, 1, 0, 0, 1, 1] 0.837037037037037
------------------------------------------------------------
gen =  6
best_off_spring of the generation 